In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyxlsb

In [ ]:
import numpy as np
import pandas as pd
import pyxlsb
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
pd.set_option('display.max_colwidth', 1000)

In [ ]:
%%time
DATASET_PATH = ""
data_AS = pd.read_excel(DATASET_PATH, sheet_name = "AS", engine='pyxlsb')
data_HB = pd.read_excel(DATASET_PATH, sheet_name = "HB", engine='pyxlsb')
data_DRV = pd.read_excel(DATASET_PATH, sheet_name = "DRV", engine='pyxlsb')

## Orders Completed ,Cancelled and Returned

In [ ]:
Total_orders =     data_AS['Order ID'].nunique()
order_completed =  data_AS[(data_AS['Order Status']=='Completed') | (data_AS['Order Status']=='Pickup Scheduled')
                      | (data_AS['Order Status']=='Processing') | (data_AS['Order Status']=='Ready to Process') | (data_AS['Order Status']=='Ready to Ship')
                      | (data_AS['Order Status']=='Shipdelight') | (data_AS['Order Status']=='Shipped') | (data_AS['Order Status']=='Shiprocket')].shape[0]
order_cancelled =  data_AS[(data_AS['Order Status']=='Cancelled') | (data_AS['Order Status']=='Failed') | (data_AS['Order Status']=='Not Serviceable') |
                           (data_AS['Order Status']=='On hold') | (data_AS['Order Status']=='Pending payment')].shape[0]
order_returned =   data_AS[(data_AS['Order Status']=='Refunded') | (data_AS['Order Status']=='RTO DELIVERED')].shape[0]

In [ ]:
Order_Status_df = pd.DataFrame(data=[Total_orders, order_completed,order_cancelled, order_returned],
                                  index = ["Total orders","Orders completed", "Order cancelled", "Order Returned"],
                               columns = ['orders'])
Order_Status_df['Percentage'] = round(Order_Status_df['orders'] *100 /Total_orders,2)

In [ ]:
Order_Status_df

## Customers with only 1 order, customers with more than 2,3,4,5 orders

In [ ]:
df = pd.DataFrame(data_AS['Email (Billing)'].value_counts()).reset_index(drop=False)
df.rename(columns = {"index":"Email_ID","Email (Billing)" : "Count" }, inplace = True)

In [ ]:
for i in np.arange(5):
  print("No. of Customers with only {} order/orders : {}".format(i+1,len(df[df['Count']==i+1])))

In [ ]:
print("No. of Customers with 2 or more orders : ",len(df[df['Count']>=2]))
print("No. of Customers with 3 or more orders : ",len(df[df['Count']>=3]))
print("No. of Customers with 4 or more orders : ",len(df[df['Count']>=4]))
print("No. of Customers with 5 or more orders : ",len(df[df['Count']>=5]))

In [ ]:
# Exporting Files to CSV
customer_with_5_and_more = df[df['Count']>=5]
output =data_AS[data_AS['Email (Billing)'].isin(customer_with_5_and_more['Email_ID'])]
print(len(output['Email (Billing)'].unique()))
output.to_csv("", index= False)

## What is the median Average Order Value? Put customers in three buckets based on AOV - Low, Median, High


In [ ]:
median_order_value = data_AS['Order Total Amount'].median()

def get_AOV_status(x):
  if (x < median_order_value):
    return "Low AOV Customer"
  elif (x == median_order_value):
    return "Medium AOV Customer"
  elif (x > median_order_value):
    return "High AOV Customer"

data_AS_AOV = pd.DataFrame(data_AS.groupby('Email (Billing)')['Order Total Amount'].sum()).reset_index(drop=False)
data_AS_AOV['AOV'] = data_AS_AOV['Order Total Amount'].apply(lambda x : get_AOV_status(x))
print("Median order value : ", median_order_value)

In [ ]:
AOV_status = pd.DataFrame(data_AS_AOV['AOV'].value_counts().sort_values()).reset_index(drop=False)
AOV_status.rename(columns = {"index" :"Category","AOV":"No. Of Customers"}, inplace= True)
AOV_status

In [ ]:
# High AOV Customers Dataset Exporting to CSV
high_AOV_customers = data_AS_AOV[data_AS_AOV['AOV']== "High AOV Customer"]
high_AOV_customers.to_csv("/content/drive/MyDrive/Google Documents/UpWork_Task/Upwork_Task_Updation/AS_High_AOV_Customers.csv", index = True)

## The lifetime value of all customers


In [ ]:
customer_lifetimne_value = pd.DataFrame(data_AS.groupby(['First Name (Billing)','Email (Billing)','Phone (Billing)']).agg({"Item #":'sum',"Order Total Amount":'sum'}))
customer_lifetimne_value["AOV"] = customer_lifetimne_value['Order Total Amount'] / customer_lifetimne_value['Item #']
customer_lifetimne_value = pd.DataFrame(customer_lifetimne_value).reset_index(drop= False)
data_AS_AOV_copy = data_AS_AOV[["Email (Billing)","AOV"]]
customer_lifetimne_value = customer_lifetimne_value.merge(data_AS_AOV_copy,on = "Email (Billing)")
customer_lifetimne_value.rename(columns ={"AOV_x":"AOV","AOV_y":"AOV Status"}, inplace= True)
customer_lifetimne_value.to_csv("/content/drive/MyDrive/Google Documents/UpWork_Task/Upwork_Task_Updation/AS_customer_lifetime_value.csv", index = True)
print("LifeTime Value of all Customers : " ,np.average(customer_lifetimne_value['Order Total Amount']))

In [ ]:
customer_lifetimne_value

## What products are frequently bought together in the same order?

In [ ]:
# Getting Unique Products
def get_unique_items(x):
  unique_values = []
  all_values = list(set((x.split("'"))))
  for i in np.arange(len(all_values)):
    if(all_values[i]) == ", ":
      continue
    elif(all_values[i]) == "":
      continue
    else:
      unique_values.append(all_values[i])
  return unique_values

frequently_bought_products = pd.DataFrame(data_AS.groupby('Item Name')['Item Name'].count()).rename(columns={"Item Name": "No. of Times Purchased"}).reset_index(drop=False)
frequently_bought_products['No.of Items'] = frequently_bought_products['Item Name'].str.count("',") +1
frequently_bought_products.sort_values(by='No.of Items', ascending = False, inplace = True)
frequently_bought_products = frequently_bought_products[frequently_bought_products['No.of Items']>=2].reset_index(drop=True)


frequently_bought_products["Splitted_Item_Name"] = frequently_bought_products['Item Name'].str.split(",'", expand=True).replace("'",'')
frequently_bought_products['Unique_Item_Name']  = frequently_bought_products['Splitted_Item_Name'].apply(lambda x : get_unique_items(x))
frequently_bought_products['Unique_Item_Name_Count'] = frequently_bought_products['Unique_Item_Name'].str.len()
frequently_bought_products.drop(['Splitted_Item_Name'], axis=1, inplace = True)
frequently_bought_products.to_csv("/content/drive/MyDrive/Google Documents/UpWork_Task/Upwork_Task_Updation/AS_frequently_bought_Unique_products.csv", index=True)

In [ ]:
frequently_bought_products

## What products are bought by the same person in subsequent purchases?


In [ ]:
%%time
DATASET_PATH = "/content/drive/MyDrive/Google Documents/UpWork_Task/Upwork_Task_Updation/Dump - 2021.xlsb"
data_AS_1 = pd.read_excel(DATASET_PATH, sheet_name = "AS", engine='pyxlsb')

In [ ]:
# Sort data as per order ID
data_AS_1 = data_AS_1.sort_values(by="Order ID", ascending = True).reset_index(drop=True)

In [ ]:
# Get Combination of All Two Lists
First_Time_Purchased_Item = data_AS_1['Item Name'].unique()
Second_Time_Purchased_Item = data_AS_1['Item Name'].unique()
cross_selling = pd.DataFrame(list(product(First_Time_Purchased_Item, Second_Time_Purchased_Item)), columns=['First_Time_Purchased_Item', 'Second_Time_Purchased_Item'])
cross_selling['instance'] = 0


unique_email_ids = data_AS_1['Email (Billing)'].unique()
for i in np.arange(len(unique_email_ids)):
  unique_orders = data_AS_1[data_AS_1["Email (Billing)"] == unique_email_ids[i]]['Order ID'].unique().tolist()
  if(len(unique_orders)>1):
    for j in np.arange(len(unique_orders)):
      previous_order_items = data_AS_1[data_AS_1['Order ID'] == unique_orders[j]]['Item Name'].unique().tolist()
      all_next_orders_items = data_AS_1[data_AS_1['Order ID'].isin(unique_orders[j+1:len(unique_orders)])]['Item Name'].unique().tolist()
      temp_df = pd.DataFrame(list(product(previous_order_items, all_next_orders_items)), columns=['previous_order_items', 'all_next_orders_items'])
      for p in np.arange(len(temp_df)):
        first_value = temp_df['previous_order_items'][p]
        second_value = temp_df['all_next_orders_items'][p]
        if(first_value != second_value):
            index = cross_selling.index[(cross_selling['First_Time_Purchased_Item']==first_value) & (cross_selling['Second_Time_Purchased_Item']==second_value)]
            cross_selling['instance'][index[0]] += 1

In [ ]:
# Exorting to CSV
cross_selling.to_csv("/content/drive/MyDrive/Google Documents/UpWork_Task/Upwork_Task_Updation/Subsequent_Purchase.csv", index= False)

## What is the customer overlap between the three sites?

In [ ]:
# AS HB DRV
unique_customers_AS = data_AS['Email (Billing)'].unique().tolist()
unique_customers_HB = data_HB['Email (Billing)'].unique().tolist()
unique_customers_DRV = data_DRV['Email (Billing)'].unique().tolist()

print("Unique Customers in AS :",len(unique_customers_AS))
print("Unique Customers in HB :",len(unique_customers_HB))
print("Unique Customers in DRV :",len(unique_customers_DRV))

In [ ]:
AS_and_HB_customers = set(unique_customers_AS) & set(unique_customers_HB)
AS_and_DRV_customers = set(unique_customers_AS) & set(unique_customers_DRV)
GB_and_DRV_customers = set(unique_customers_HB) & set(unique_customers_DRV)
GB_and_DRV_and_AS_customers =  set(unique_customers_HB) & set(unique_customers_DRV) & set(unique_customers_AS)

print("Customers who are in both AS and HB :", len(AS_and_HB_customers))
print("Customers who are in both AS and DRV :", len(AS_and_DRV_customers))
print("Customers who are in both GB and DRV :", len(GB_and_DRV_customers))
print("Customers who are in all three sites :", len(GB_and_DRV_and_AS_customers))